In [7]:
import pandas as pd
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import requests
import plotly.express as px

pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)
pd.set_option('display.float_format', lambda x: f'%.{2}f' % x)
pd.set_option('display.max_colwidth', None)

In [8]:
url = "https://tfl.gov.uk/tfl/syndication/feeds/cycle-hire/livecyclehireupdates.xml"
response = requests.get(url)
data = response.content

In [9]:
df1 = pd.read_xml(data)
df1.drop(['id', 'terminalName', 'installed', 'locked',
       'installDate', 'removalDate', 'temporary'], axis=1, inplace=True)
df1.head()

,name,lat,long,nbBikes,nbStandardBikes,nbEBikes,nbEmptyDocks,nbDocks
0,"River Street , Clerkenwell",51.53,-0.11,2,2,0,17,19
1,"Phillimore Gardens, Kensington",51.50,-0.20,16,16,0,21,37
2,"Christopher Street, Liverpool Street",51.52,-0.08,2,2,0,30,32
3,"St. Chad's Street, King's Cross",51.53,-0.12,15,15,0,8,23
4,"Sedding Street, Sloane Square",51.49,-0.16,25,25,0,2,27


In [10]:
df2 = pd.read_csv('../8Jul2020-14Jul2020.csv')
# df2.drop(['EndStation Lat', 'EndStation Lng', 'StartStation Lat',
#        'StartStation Lng', 'EndStation numOfBikes',
#        'EndStation numOfStandardBikes', 'EndStation numOfEBikes',
#        'EndStation numOfEmptyDocks', 'EndStation numOfDocks',
#        'StartStation numOfBikes', 'StartStation numOfStandardBikes',
#        'StartStation numOfEBikes', 'StartStation numOfEmptyDocks',
#        'StartStation numOfDocks'], inplace=True, axis=1)
df2.head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,EndStation lat,EndStation long,EndStation nbBikes,EndStation nbStandardBikes,EndStation nbEBikes,EndStation nbEmptyDocks,EndStation nbDocks,StartStation lat,StartStation long,StartStation nbBikes,StartStation nbStandardBikes,StartStation nbEBikes,StartStation nbEmptyDocks,StartStation nbDocks
0,99188805,780,13656,08/07/2020 00:13,476,"Stebondale Street, Cubitt Town",08/07/2020 00:00,532,"Jubilee Plaza, Canary Wharf",51.49,-0.01,26,24,2,3,29,51.50,-0.02,46,45,1,17,63
1,99188806,180,4393,08/07/2020 00:03,531,"Twig Folly Bridge, Mile End",08/07/2020 00:00,517,"Ford Road, Old Ford",51.53,-0.04,6,6,0,15,21,51.53,-0.03,2,2,0,16,18
2,99188809,960,8973,08/07/2020 00:17,508,"Fournier Street, Whitechapel",08/07/2020 00:01,779,"Houndsditch, Aldgate",51.52,-0.07,8,8,0,10,18,51.51,-0.08,3,3,0,25,28
3,99188810,960,7141,08/07/2020 00:17,508,"Fournier Street, Whitechapel",08/07/2020 00:01,779,"Houndsditch, Aldgate",51.52,-0.07,8,8,0,10,18,51.51,-0.08,3,3,0,25,28
4,99188807,600,15085,08/07/2020 00:11,201,"Dorset Square, Marylebone",08/07/2020 00:01,797,"Ossulston Street, Somers Town",51.52,-0.16,15,15,0,1,16,51.53,-0.13,28,27,1,2,30


In [11]:
df = df2.merge(df1, left_on='EndStation Name', right_on='name')
df.drop(['name'], inplace=True, axis=1)
df.columns.values[9:] = 'EndStation ' + df.columns.values[9:]


df = df.merge(df1, left_on='StartStation Name', right_on='name')
df.drop(['name'], inplace=True, axis=1)
df.columns.values[16:] = 'StartStation ' + df.columns.values[16:]

df.set_index(['Rental Id'], inplace=True)
df.head()

,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,EndStation EndStation lat,EndStation EndStation long,EndStation EndStation nbBikes,EndStation EndStation nbStandardBikes,EndStation EndStation nbEBikes,EndStation EndStation nbEmptyDocks,EndStation EndStation nbDocks,StartStation EndStation StartStation lat,StartStation EndStation StartStation long,StartStation EndStation StartStation nbBikes,StartStation EndStation StartStation nbStandardBikes,StartStation EndStation StartStation nbEBikes,StartStation EndStation StartStation nbEmptyDocks,StartStation EndStation StartStation nbDocks,StartStation EndStation lat,StartStation EndStation long,StartStation EndStation nbBikes,StartStation EndStation nbStandardBikes,StartStation EndStation nbEBikes,StartStation EndStation nbEmptyDocks,StartStation EndStation nbDocks,StartStation lat,StartStation long,StartStation nbBikes,StartStation nbStandardBikes,StartStation nbEBikes,StartStation nbEmptyDocks,StartStation nbDocks
Rental Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
99188805,780,13656,08/07/2020 00:13,476,"Stebondale Street, Cubitt Town",08/07/2020 00:00,532,"Jubilee Plaza, Canary Wharf",51.49,-0.01,26,24,2,3,29,51.50,-0.02,46,45,1,17,63,51.49,-0.01,20,20,0,9,29,51.50,-0.02,47,47,0,16,63
99294025,960,3793,11/07/2020 13:42,476,"Stebondale Street, Cubitt Town",11/07/2020 13:26,532,"Jubilee Plaza, Canary Wharf",51.49,-0.01,26,24,2,3,29,51.50,-0.02,46,45,1,17,63,51.49,-0.01,20,20,0,9,29,51.50,-0.02,47,47,0,16,63
99294024,960,16118,11/07/2020 13:42,476,"Stebondale Street, Cubitt Town",11/07/2020 13:26,532,"Jubilee Plaza, Canary Wharf",51.49,-0.01,26,24,2,3,29,51.50,-0.02,46,45,1,17,63,51.49,-0.01,20,20,0,9,29,51.50,-0.02,47,47,0,16,63
99454764,660,15235,14/07/2020 23:19,476,"Stebondale Street, Cubitt Town",14/07/2020 23:08,532,"Jubilee Plaza, Canary Wharf",51.49,-0.01,26,24,2,3,29,51.50,-0.02,46,45,1,17,63,51.49,-0.01,20,20,0,9,29,51.50,-0.02,47,47,0,16,63
99387242,1140,16422,13/07/2020 09:08,552,"Watney Street, Shadwell",13/07/2020 08:49,532,"Jubilee Plaza, Canary Wharf",51.51,-0.06,18,17,1,9,27,51.50,-0.02,46,45,1,17,63,51.51,-0.06,4,4,0,23,27,51.50,-0.02,47,47,0,16,63


In [12]:
# df.to_csv('test.csv')